# 04 - Pipeline Final e Geração da Submissão

**🎯 PROPÓSITO DESTE NOTEBOOK:**
Este notebook contém o pipeline final para o Hackathon Forecast Big Data 2025. O processo consiste em:

1. **Carregamento dos Dados Brutos:** Carregar os dados de 2022 (transações, produtos, pdvs)
2. **Engenharia de Features:** Função completa para processar os dados e criar features
3. **Treinamento do Modelo Final:** Treinar o LightGBM com 100% dos dados de 2022
4. **Geração do Grid de Previsão:** Criar o dataframe base para as 5 semanas de Janeiro/2023
5. **Previsão e Submissão:** Gerar previsões e salvar arquivos CSV e Parquet

**🚀 PIPELINE COMPLETO:**
Este notebook executa todo o processo de ponta a ponta, da engenharia de features até a geração da submissão final.

In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
import lightgbm as lgb
from datetime import datetime, timedelta
import gc
import os

# Use o Polars para a engenharia de features, como no notebook 02
import polars as pl

warnings.filterwarnings('ignore')
print('📚 Bibliotecas carregadas com sucesso!')
print('🎯 Iniciando Pipeline Final para Submissão')

📚 Bibliotecas carregadas com sucesso!
🎯 Iniciando Pipeline Final para Submissão


In [ ]:
def engenharia_de_features(df_transacoes, df_produtos, df_pdvs):
    """
    Aplica a engenharia de features completa usando Polars para eficiência.
    Recebe os dataframes brutos e retorna um dataframe pandas com as features.
    
    Esta função implementa EXATAMENTE a mesma lógica do notebook 02-Feature-Engineering-Dask.ipynb
    """
    print("🔧 Iniciando engenharia de features com Polars...")
    
    # Converter para Polars para performance
    transacoes_pl = pl.from_pandas(df_transacoes)
    produtos_pl = pl.from_pandas(df_produtos)
    pdvs_pl = pl.from_pandas(df_pdvs)
    
    print(f"   • Transações: {len(transacoes_pl):,} registros")
    print(f"   • Produtos: {len(produtos_pl):,} registros")
    print(f"   • PDVs: {len(pdvs_pl):,} registros")
    
    # === INÍCIO DA LÓGICA ADAPTADA DO NOTEBOOK 02 ===
    
    # 1. Renomear colunas para consistência (CORREÇÃO BASEADA NOS DADOS REAIS)
    print("📝 Renomeando colunas para consistência...")
    
    # Transações: renomear para nomes padronizados
    transacoes_pl = transacoes_pl.rename({
        'internal_product_id': 'produto_id',
        'internal_store_id': 'pdv_id',
        'reference_date': 'semana',
        'quantity': 'quantidade'
    })
    
    # Produtos: usar as colunas que existem realmente
    produtos_pl = produtos_pl.rename({
        'produto': 'produto_id'  # A coluna chave é 'produto', não 'internal_product_id'
    })
    
    # PDVs: usar as colunas que existem realmente  
    pdvs_pl = pdvs_pl.rename({
        'pdv': 'pdv_id'  # A coluna chave é 'pdv', não 'internal_store_id'
    })
    
    print("✅ Colunas renomeadas com sucesso!")
    
    # 2. Joins dos dados (agora vai funcionar)
    print("🔗 Fazendo joins dos dados...")
    dados = transacoes_pl.join(produtos_pl, on='produto_id', how='left').join(pdvs_pl, on='pdv_id', how='left')
    
    # 3. Conversão de data e ordenação
    print("📅 Processando datas...")
    dados = dados.with_columns(pl.col("semana").str.to_datetime("%Y-%m-%d"))
    dados = dados.sort(["pdv_id", "produto_id", "semana"])
    
    # 4. Features temporais (mês, semana do ano, sin/cos para sazonalidade)
    print("🕒 Criando features temporais...")
    dados = dados.with_columns([
        pl.col("semana").dt.month().alias("mes"),
        pl.col("semana").dt.week().alias("semana_ano")
    ])
    
    # Features cíclicas para capturar sazonalidade
    dados = dados.with_columns([
        (pl.col("mes") * (2 * np.pi / 12)).sin().alias("mes_sin"),
        (pl.col("mes") * (2 * np.pi / 12)).cos().alias("mes_cos")
    ])
    
    # 5. Features de Lag (valores passados)
    print("⏪ Criando features de lag...")
    lags = [1, 2, 3, 4]
    for lag in lags:
        dados = dados.with_columns(
            pl.col("quantidade").shift(lag).over(["pdv_id", "produto_id"]).alias(f"quantidade_lag_{lag}")
        )
    
    # 6. Features de Rolling Window (médias móveis, etc.)
    print("📊 Criando features de rolling window...")
    dados = dados.with_columns([
        pl.col("quantidade").rolling_mean(window_size=4, min_periods=1).over(["pdv_id", "produto_id"]).alias("quantidade_media_4w"),
        pl.col("quantidade").rolling_max(window_size=4, min_periods=1).over(["pdv_id", "produto_id"]).alias("quantidade_max_4w"),
        pl.col("quantidade").rolling_min(window_size=4, min_periods=1).over(["pdv_id", "produto_id"]).alias("quantidade_min_4w")
    ])
    
    # 7. Features de Hash para reduzir dimensionalidade
    print("🔢 Criando features de hash...")
    dados = dados.with_columns([
        pl.col("pdv_id").hash().cast(pl.Int8).alias("pdv_hash"),
        pl.col("produto_id").hash().cast(pl.Int8).alias("produto_hash"),
        (pl.col("pdv_id").cast(str) + "_" + pl.col("produto_id").cast(str)).hash().cast(pl.Int16).alias("pdv_produto_hash")
    ])
    
    # 8. Features históricas por combinação PDV/Produto
    print("📈 Criando features históricas...")
    dados = dados.with_columns([
        pl.col("quantidade").mean().over(["pdv_id", "produto_id"]).alias("hist_mean"),
        pl.col("quantidade").std().over(["pdv_id", "produto_id"]).alias("hist_std"),
        pl.col("quantidade").max().over(["pdv_id", "produto_id"]).alias("hist_max"),
        pl.col("quantidade").count().over(["pdv_id", "produto_id"]).cast(pl.Int8).alias("hist_count")
    ])
    
    # 9. Preencher NaNs que surgiram dos lags/rolling
    print("🔧 Preenchendo valores missing...")
    dados = dados.fill_null(0)
    
    # === FIM DA LÓGICA ADAPTADA DO NOTEBOOK 02 ===
    
    print("✅ Engenharia de features concluída!")
    
    # Converter de volta para pandas
    df_final = dados.to_pandas()
    print(f"   • Shape final: {df_final.shape}")
    print(f"   • Features criadas: {len(df_final.columns)}")
    
    return df_final

print("🛠️ Função de engenharia de features definida com sucesso!")

In [3]:
# Carregar dados brutos de 2022 diretamente
print("📂 Carregando dados brutos de 2022...")

# Carregar os dados brutos dos arquivos parquet originais
df_transacoes_2022 = pd.read_parquet('../data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet')
df_produtos = pd.read_parquet('../data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet')
df_pdvs = pd.read_parquet('../data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet')

print(f"   • Transações: {df_transacoes_2022.shape}")
print(f"   • Produtos: {df_produtos.shape}") 
print(f"   • PDVs: {df_pdvs.shape}")

# Aplicar engenharia de features usando a função criada
print('\n🔧 Aplicando engenharia de features...')
dados_treino_com_features = engenharia_de_features(df_transacoes_2022, df_produtos, df_pdvs)

print(f'\n📊 Dados com features processados:')
print(f'   • Shape: {dados_treino_com_features.shape}')
print(f'   • Período: {dados_treino_com_features["semana"].min()} até {dados_treino_com_features["semana"].max()}')

# Aplicar otimização de memória (downcasting) como no seu trabalho anterior
print('\n📊 Otimização de memória e tratamento de missing values...')

# Downcasting para otimizar memória
print('🔽 Aplicando downcasting...')
for col in dados_treino_com_features.select_dtypes(include=[np.number]).columns:
    original_dtype = dados_treino_com_features[col].dtype
    if dados_treino_com_features[col].dtype.kind in ['i', 'u']:  # Inteiros
        dados_treino_com_features[col] = pd.to_numeric(dados_treino_com_features[col], downcast='integer')
    else:  # Floats
        dados_treino_com_features[col] = pd.to_numeric(dados_treino_com_features[col], downcast='float')

# Otimizar colunas categóricas
print('📂 Otimizando categóricas...')
for col in dados_treino_com_features.select_dtypes(include=['object']).columns:
    if col not in ['semana']:  # Preservar datetime
        nunique = dados_treino_com_features[col].nunique()
        total_rows = len(dados_treino_com_features)
        if nunique / total_rows < 0.5:  # Se <50% valores únicos, usar category
            dados_treino_com_features[col] = dados_treino_com_features[col].astype('category')

# Tratamento inteligente de missing values para distributor_id
if 'distributor_id' in dados_treino_com_features.columns:
    print('🔧 Tratando missing values em distributor_id...')
    if dados_treino_com_features['distributor_id'].dtype.name == 'category':
        if -1 not in dados_treino_com_features['distributor_id'].cat.categories:
            dados_treino_com_features['distributor_id'] = dados_treino_com_features['distributor_id'].cat.add_categories([-1])
    dados_treino_com_features['distributor_id'] = dados_treino_com_features['distributor_id'].fillna(-1)

print('✅ Otimização concluída!')

# Definir features e target
target = 'quantidade'
exclude_features = [
    'pdv_id', 'produto_id', 'semana', 'quantidade',  # IDs e target
    'valor', 'num_transacoes',  # Features que vazam informação do futuro (se existirem)
    'mes', 'semana_ano'  # Features temporais brutas (mantemos sin/cos)
]

all_features = [col for col in dados_treino_com_features.columns if col not in exclude_features]

print(f'\n🎯 Preparando dados para treinamento:')
print(f'   • Target: {target}')
print(f'   • Features disponíveis: {len(all_features)}')
print(f'   • Features excluídas: {len(exclude_features)}')

X_full = dados_treino_com_features[all_features]
y_full = dados_treino_com_features[target]

print(f'   • X_full shape: {X_full.shape}')
print(f'   • y_full shape: {y_full.shape}')

# Treinamento do modelo final
print('\n🚀 Treinando o modelo LightGBM final com todos os dados de 2022...')

lgb_params_final = {
    'objective': 'regression_l1',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'random_state': 42,
    'n_jobs': -1
}

best_iteration = 200  # Usar a melhor iteração da validação anterior

train_full_lgb = lgb.Dataset(X_full, label=y_full, free_raw_data=False)  # free_raw_data=False para reutilização
final_model = lgb.train(lgb_params_final, train_full_lgb, num_boost_round=best_iteration)

print(f'✅ Modelo final treinado com sucesso em {best_iteration} iterações!')
print(f'   • Features utilizadas: {len(all_features)}')
print(f'   • Lista de features: {all_features[:10]}...')  # Mostrar primeiras 10

# Limpeza parcial de memória (manter dados necessários para teste)
print('\n🧹 Limpeza de memória...')
del train_full_lgb
gc.collect()

print('🎉 Pipeline de treinamento concluído com sucesso!')

📂 Carregando dados brutos de 2022...
   • Transações: (6560698, 11)
   • Produtos: (7092, 8)
   • PDVs: (14419, 4)

🔧 Aplicando engenharia de features...
🔧 Iniciando engenharia de features com Polars...
   • Transações: 6,560,698 registros
   • Produtos: 7,092 registros
   • PDVs: 14,419 registros
📝 Renomeando colunas para consistência...


ColumnNotFoundError: "internal_product_id" not found

In [ ]:
print('📅 Preparando dados para as previsões de Janeiro/2023...')

# 1. Criar o Grid de Teste para as 5 semanas de Janeiro/2023
print('🎯 Criando grid de teste para Janeiro/2023...')

# CORREÇÃO: Usar os nomes originais das colunas dos dados brutos
unique_combinations = df_transacoes_2022[['internal_store_id', 'internal_product_id']].drop_duplicates()
unique_combinations.rename(columns={'internal_store_id': 'pdv_id', 'internal_product_id': 'produto_id'}, inplace=True)

semanas_2023 = pd.DataFrame({'semana_int': range(1, 6)})
df_teste_base = unique_combinations.merge(semanas_2023, how='cross')

# Converter semana_int para formato de data (começando em 2023-01-03, primeira terça de janeiro)
df_teste_base['semana'] = df_teste_base['semana_int'].apply(
    lambda s: pd.to_datetime('2023-01-03') + pd.to_timedelta(s-1, unit='W')
)

# Adicionar quantidade = 0 como placeholder (necessário para a função de features)
df_teste_base['quantidade'] = 0

print(f'   • Grid de teste: {df_teste_base.shape}')
print(f'   • Combinações únicas: {len(unique_combinations):,}')
print(f'   • Semanas: {sorted(df_teste_base["semana_int"].unique())}')
print(f'   • Período: {df_teste_base["semana"].min()} a {df_teste_base["semana"].max()}')

# 2. ESTRATÉGIA CRUCIAL: Combinar histórico + grid de teste para features contínuas
print('\n🔧 Combinando dados históricos com grid de teste...')
print('   💡 Estratégia: Esta combinação permite calcular lags e médias móveis corretas')

# Preparar dados históricos com nomes de colunas consistentes
df_historico = df_transacoes_2022[['reference_date', 'internal_store_id', 'internal_product_id', 'quantity']].copy()
df_historico.rename(columns={
    'reference_date': 'semana',
    'internal_store_id': 'pdv_id', 
    'internal_product_id': 'produto_id',
    'quantity': 'quantidade'
}, inplace=True)

# Garantir que as colunas sejam consistentes entre histórico e teste
colunas_necessarias = ['semana', 'pdv_id', 'produto_id', 'quantidade']
df_teste_grid = df_teste_base[colunas_necessarias].copy()

# Combinar dados históricos de 2022 + grid de teste de 2023
dados_para_features_teste = pd.concat([df_historico, df_teste_grid], ignore_index=True)
dados_para_features_teste = dados_para_features_teste.sort_values(['pdv_id', 'produto_id', 'semana']).reset_index(drop=True)

print(f'   • Dados históricos: {len(df_historico):,}')
print(f'   • Grid de teste: {len(df_teste_grid):,}')
print(f'   • Dados combinados: {dados_para_features_teste.shape}')

# 3. Reutilizar a função de engenharia de features!
print('\n🛠️ Aplicando engenharia de features aos dados combinados...')
dados_teste_com_features = engenharia_de_features(dados_para_features_teste, df_produtos, df_pdvs)

print(f'   • Dados com features: {dados_teste_com_features.shape}')

# 4. Filtrar apenas as semanas de 2023 para a previsão
print('\n🎯 Filtrando dados de teste (Janeiro/2023)...')
mask_2023 = dados_teste_com_features['semana'].dt.year == 2023
X_teste = dados_teste_com_features[mask_2023].copy()

print(f'   • Dados de teste filtrados: {X_teste.shape}')
print(f'   • Verificação: {X_teste["semana"].min()} a {X_teste["semana"].max()}')

# 5. Aplicar mesma otimização de memória e tratamento de missing no X_teste
print('\n📊 Aplicando otimização aos dados de teste...')

# Downcasting
print('🔽 Downcasting dados de teste...')
for col in X_teste.select_dtypes(include=[np.number]).columns:
    if X_teste[col].dtype.kind in ['i', 'u']:  # Inteiros
        X_teste[col] = pd.to_numeric(X_teste[col], downcast='integer')
    else:  # Floats
        X_teste[col] = pd.to_numeric(X_teste[col], downcast='float')

# Otimizar categóricas
print('📂 Otimizando categóricas teste...')
for col in X_teste.select_dtypes(include=['object']).columns:
    if col not in ['semana']:
        nunique = X_teste[col].nunique()
        total_rows = len(X_teste)
        if nunique / total_rows < 0.5:
            X_teste[col] = X_teste[col].astype('category')

# Tratamento de missing values
if 'distributor_id' in X_teste.columns:
    print('🔧 Tratando missing values em distributor_id (teste)...')
    if X_teste['distributor_id'].dtype.name == 'category':
        if -1 not in X_teste['distributor_id'].cat.categories:
            X_teste['distributor_id'] = X_teste['distributor_id'].cat.add_categories([-1])
    X_teste['distributor_id'] = X_teste['distributor_id'].fillna(-1)

print('✅ Otimização dos dados de teste concluída!')

# 6. Preparar features para previsão (garantir mesma ordem do treino)
print('\n🔮 Preparando features para previsão...')
features_teste = X_teste[all_features]

print(f'   • Shape das features de teste: {features_teste.shape}')
print(f'   • Features esperadas: {len(all_features)}')
print(f'   • Verificação de consistência: {list(features_teste.columns) == all_features}')

# 7. FAZER A PREVISÃO REAL (não mais placeholder!)
print('\n🚀 Gerando previsões REAIS com o modelo treinado...')
predictions = final_model.predict(features_teste)
predictions = np.maximum(0, predictions)  # Não permitir previsões negativas

print(f'   • Previsões geradas: {len(predictions):,}')
print(f'   • Estatísticas das previsões:')
print(f'     - Mínimo: {predictions.min():.2f}')
print(f'     - Máximo: {predictions.max():.2f}')
print(f'     - Média: {predictions.mean():.2f}')
print(f'     - Mediana: {np.median(predictions):.2f}')
print(f'     - Zeros: {(predictions == 0).sum():,} ({(predictions == 0).mean()*100:.1f}%)')

# 8. Montar o arquivo de submissão
print('\n📋 Montando arquivo de submissão...')
df_submissao = X_teste[['semana', 'pdv_id', 'produto_id']].copy()

# Mapear a semana datetime de volta para o número da semana (1-5)
week_map = {d: i+1 for i, d in enumerate(sorted(df_submissao['semana'].unique()))}
df_submissao['semana'] = df_submissao['semana'].map(week_map)

# Adicionar previsões (arredondadas para inteiros)
df_submissao['quantidade'] = predictions.round().astype(int)

# Renomear colunas para formato de submissão
df_submissao.rename(columns={'pdv_id': 'pdv', 'produto_id': 'produto'}, inplace=True)

print(f'   • Formato final: {df_submissao.shape}')
print(f'   • Colunas: {list(df_submissao.columns)}')

# 9. Salvar arquivos de submissão
print('\n💾 Salvando arquivos de submissão...')
os.makedirs('../submissions', exist_ok=True)

# Salvar em Parquet
caminho_parquet = '../submissions/previsao_final.parquet'
df_submissao.to_parquet(caminho_parquet, index=False)
print(f'✅ Arquivo Parquet salvo: {caminho_parquet}')

# Salvar em CSV
caminho_csv = '../submissions/previsao_final.csv'
df_submissao.to_csv(caminho_csv, index=False, sep=';', encoding='utf-8')
print(f'✅ Arquivo CSV salvo: {caminho_csv}')

# 10. Mostrar amostra e estatísticas finais
print('\n📋 Amostra da submissão final:')
print(df_submissao.head(15))

print('\n🎉 PIPELINE COMPLETO - SUBMISSÃO GERADA COM SUCESSO!')
print('=' * 70)
print(f'📊 Estatísticas finais da submissão:')
print(f'   • Total de previsões: {len(df_submissao):,}')
print(f'   • Semanas cobertas: {sorted(df_submissao["semana"].unique())}')
print(f'   • Combinações únicas (PDV×Produto): {df_submissao.groupby(["pdv", "produto"]).size().count():,}')
print(f'   • Estatísticas das quantidades previstas:')
print(f'     - Total previsto: {df_submissao["quantidade"].sum():,}')
print(f'     - Média por previsão: {df_submissao["quantidade"].mean():.2f}')
print(f'     - Previsões não-zero: {(df_submissao["quantidade"] > 0).sum():,} ({(df_submissao["quantidade"] > 0).mean()*100:.1f}%)')

print(f'\n📁 Arquivos de submissão gerados:')
print(f'   • CSV: {caminho_csv}')
print(f'   • Parquet: {caminho_parquet}')

print('\n🏆 PIPELINE PONTA A PONTA CONCLUÍDO!')
print('   ✅ Dados brutos → Features → Modelo → Previsões → Submissão')
print('   ✅ Nenhuma dependência externa ou arquivo pré-processado')
print('   ✅ Pronto para avaliação!')